<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Data preprocessing</center>

# Extra worksheet S7: Introducción a Word Embeddings

Los Word Embeddings llevan mucho tiempo existiendo. De hecho, ya hemos hablado de ellos: Bag of Words podría considerarse un Word Embedding donde una palabra o término es representada por su aparición o no aparición en los documentos del dataset. Sin embargo, y pese a que funciona muy bien para problemas pequeños y específicos tiene varios problemas:
* Matrices gigantescas que consumen la memoria RAM.

In [6]:
import numpy as np
matriz = np.random.rand(8000, 20000)
print("Memoria: {} GB".format(matriz.nbytes/10**9))

Memoria: 1.28 GB


* Bag of Words trata cada término como una unidad de información. No considera el contexto (aunque hay formas de intentarlo...).
* Debido a esto, cada problema es extremadamente dependiente del dominio ("temática" que esté tratando el dataset).

### ¿Qué son exactamente los Word Embeddings?

No son más que representación de términos o palabras mediante arrays o vectores numéricos. Por ejemplo `"banana"` podría estar representada por el embedding de 2 dimensiones `[0.98, 0.02]`. Estos valores podrían significar que la palabra banana tiene un 0.90 de característica de "fruta" y un 0.02 de característica de "humano", como si se tratasen de variables de un dataset habitual.

Sin embargo, **tanto el significado de estas variables como sus valores se computan automáticamente** mediante técnicas de Deep Learning, gracias a que somos capaces de capturar el contexto que rodea a cada una de las palabras.

### ¿Cómo aprende un algoritmo los Word Embeddings?

<img src="model.png" width = 800>

Para aprender los Word Embeddings, hay varios elementos clave que debemos tener en cuenta:

* Definimos un **vocabulario $V$** como la lista de palabras de nuestro corpus. Tamaño: `10000 palabras`. $$V = [a, aaron, abacus..., zulu]$$
<br>
* Cada palabra, vendrá representada por un **vector One-Hot encoding** ($o$) de tamaño `(1, longitud(V))`. P. ej: si la palabra `"banana"` es la palabra número 1231 de nuestro vocabulario, el vector One-Hot encoding $o_{1231}$ estará formado por 10000 elementos a valor 0, menos el de la posición 1231 que será un 1. Tamaño: `(1, 10000)`.$$"banana" = o_{1231} = [0, 0, 0, ..., 1, 0, ..., 0]$$
<br>
* **Matriz E** de valores (también llamados pesos o parámetros) que se irá "ajustando" al objetivo a optimizar del algoritmo. En realidad, el objetivo del algoritmo no es importante, lo importante es cómo esa matriz E se va ajustando. El tamaño de la matriz será `(longitud(V), #num. variables)`. Tamaño: `(10000, 300)`.
<br>
* **Vector e** para cada palabra. Esto es el word embedding. Tamaño: `(1, #num. variables)`.

Imaginemos ahora un problema de clasificación donde tenemos la frase _"I want a glass of orange ..."_. Obviamente esperaríamos encontrarnos _"juice"_ como palabra más probable al final de esa frase. En este caso, estamos hablando de que _"I want a glass of orange"_ es el contexto (c), y _"juice"_ es la palabra objetivo (t).

Con suerte y después de muchas iteraciones, nuestro algoritmo optimizará los pesos de la matriz E, que resultará en la obtención de nuestros Word Embeddings para todo el dataset con el que hayamos alimentado nuestro modelo. A grandes rasgos, nuestro algoritmo descubrirá que si aprende similares valores para todas las frutas, se ajustará mejor a lo que esperamos del clasificador.

### ¿Qué los hace interesantes?

* La multiplicación de matrices resultará en Word Embeddings de dimensiones de: (1, 10000) * (10000, 300) = (1, 300). Esto es mucho mejor que el sistema que hemos visto en Bag of Words, donde cada palabra dependía del número de documentos y no de un número de variables fijo.

In [12]:
import numpy as np
matriz = np.random.rand(10000, 300)
print("Memoria: {} GB".format(matriz.nbytes/10**9))

Memoria: 0.024 GB


* Gracias a ser capaces de capturar el contexto, son **capaces de generalizar** con basándose en contextos similares.
* **Transfer learning**. Una vez se han aprendido los Word Embeddings, seremos capaces de "transferirlos" a otros modelos para buscar otra utilidad sin comenzar un entrenamiento "en frío". Esto es común en Deep Learning, pero en NLP se ha demostrado muy útil.
* **Capaces de capturar correspondencias, analogías y hasta evolución en términos**:
    * Man -> Woman as King -> Queen
    * $e_{man} - e_{woman} ~= e_{king} - e_{queen}$
    * $e_{london} ~= e_{madrid} - e_{spain} + e_{uk}$

<img src="word_vectors.png" width =800>

<img src="evolving_word_embeddings.jpeg" width = 800>

### Desafíos surgidos

Uno de los desafíos surgidos a causa de esta técnica son los "bias" o "sesgos" que se asumen de aprender de un dataset junto su contexto. Si un algoritmo es entrenado con textos machistas, los Word Embeddings representarán ese machismo.

Sin embargo, hay soluciones como las que se muestran a continuación:

<img src="wordembedding_bias.png" width=800>

1. <font color='blue'>Identificar la dimensión del bias (en este caso sexo)</font>:
    1. Calculando la media de: ($e_{he} - e_{she}, e_{male} - e_{female}, ...$)
    2. El resto de dimensiones las consideramos en un único eje.
    
2. <font color='red'>Neutralizar el sesgo para todas las palabras en la que el género no sea definitorio.</font>
    1. "Doctor" y "Nurse" tienen género neutral (en inglés). Neutralizamos.
    2. "Girl" y "Boy" tienen género definitorio. No neutralizamos.
    
3. <font color='green'>Equalizar pares de palabras a la misma distancia de dimensión Bias.</font>
    1. "Grandmother" y "Grandfather" deberían estar ambas a la misma distancia del eje, pues un género no es mayor que otro.

## Librerías y aplicaciones

Las dos librerías más usadas hoy en día para realizar tareas de texto con embeddings son Gensim y Spacy. Gensim empezó siendo una libería para extraer "topics" de texto y ha acabado evolucionando a algo más complejo con embeddings como parte importante. Spacy es toda una suite de herramientas todas basadas en modelos de Deep Learning.

*Nota: Para poder instalar modelos de spacy desde jupyter notebook, hay que ejecutar jupyter notebook con permisos de administrador del equipo.*

In [90]:
# instalar requisitos
# import sys
# !{sys.executable} -m pip install spacy
# !python -m spacy download es

Las siguientes técnicas son un par de ejemplos que nos ofrecen estas librerías gracias a embeddings. Es por esto que en los próximos años será el momento en el que NLP despegará como ya lo hizo la visión por computador hace un par de años.

## Parseo de dependencias y Parts of Speech tagging

In [91]:
import spacy
from spacy import displacy
nlp = spacy.load("es")

doc = nlp(u"Esto es una frase.")
displacy.render(doc, style="dep", jupyter=True)

## NER: Named Entity Recognition

In [92]:
doc = nlp(u"Apple estaría pensando en comprar Junntar, startup de Iván Arévalo, el 1/10/2019")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
displacy.render(doc, style="ent", jupyter=True)

Apple 0 5 ORG
Junntar 34 41 MISC
Iván Arévalo 54 66 PER
1/10/2019 71 80 MISC
